8-puzzle problem

In [1]:
import random
import copy

In [2]:
# Generating solvable state
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
n_problems = 100
n_iters = 50
moves = [[-1, 0, 'Up'], [1, 0, 'Down'], [0, 1, 'Right'], [0, -1, 'Left']]
all_problems = []
rows, cols = len(goal_state), len(goal_state[0])

for _ in range(n_problems):
    new_state = copy.deepcopy(goal_state)
    # Get the position of 0
    empty_cell = 0
    for row_index, row in enumerate(new_state):
        if empty_cell in row:
            col_index = row.index(empty_cell)
            empty_r, empty_c = row_index, col_index

    
    # Shuffle
    for _ in range(n_iters): 
        dx, dy, _ = moves[random.randint(0, 3)] # 0 and 3 included
        x, y = empty_r + dx, empty_c + dy
        if 0 <= x < rows and 0 <= y < cols:
            new_state[empty_r][empty_c], new_state[x][y] =\
            new_state[x][y], new_state[empty_r][empty_c]
            empty_r, empty_c = x, y
        
    all_problems.append(copy.deepcopy(new_state))


In [3]:
# n_iters = 10000
# all_problems = []
# new_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
# goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# # Get the position of 0
# empty_cell = 0
# for row_index, row in enumerate(new_state):
#     if empty_cell in row:
#         col_index = row.index(empty_cell)
#         empty_r, empty_c = row_index, col_index

# for i in range(n_iters):
#     rand = random.randint(0, 3)
#     dx, dy, _ = moves[rand] # 0 and 3 included
#     x, y = empty_r + dx, empty_c + dy
#     if 0 <= x < rows and 0 <= y < cols:
#         new_state[empty_r][empty_c], new_state[x][y] =\
#         new_state[x][y], new_state[empty_r][empty_c]
#         empty_r, empty_c = x, y

#     if i % 10 == 0:
#         all_problems.append(copy.deepcopy(new_state))
        
# print(all_problems)

In [15]:
# Define heuristic function (Manhataen Distance is used)
def manhattan_dist(current_state, goal_state):
    """
    Args:
        current_state (matrix) : the matrix to be checked
        goal_state (matrix) : the goal matrix
    Returns:
        distance (int) : the manhattan distance between current state and goal state
    """
    distance = 0
    for row in range(len(current_state)):
        for col in range(len(current_state[0])):
            state = current_state[row][col]
            # Finding the position of state in the goal_state
            for i in range(len(goal_state)):
                if state in goal_state[i]:
                    goal_col = goal_state[i].index(state)
                    goal_row = i
            dist = abs(row - goal_row) + abs(col - goal_col)
            distance += dist

    return distance


In [16]:
# Get all possible neighbor states (states after 1 valid move)
def get_neighbors(current_state):
    """
    Args:
        current_state (narray(n, n)) : Matrix of the current state
    Returns:
        possible_next_states (List of matrix) : List of all possible next states
    """
    # Find the empty cell (cell with 0)
    empty = 0
    total_rows = len(current_state)
    total_cols = len(current_state[0])
    for i in range(total_rows):
        if empty in current_state[i]:
            empty_col = current_state[i].index(empty)
            empty_row = i

    # Define possible moves
    possible_next_states = []
    valid_moves = [[-1, 0, 'Up'], [1, 0, 'Down'], [0, 1, 'Right'], [0, -1, 'Left']]
    for dx, dy, _ in valid_moves:
        new_row = empty_row + dx
        new_col = empty_col + dy
        if 0 <= new_row < total_rows and 0 <= new_col < total_cols:
            decoy = copy.deepcopy(current_state)
            decoy[empty_row][empty_col], decoy[new_row][new_col] =\
            decoy[new_row][new_col], decoy[empty_row][empty_col]
            possible_next_states.append(decoy)
    
    return possible_next_states   

In [17]:
def hill_climbing_search(initial_state):
    """
    Main function for the search (hill climbing algorithm)
    Args:
        initial_state (narray(n, n)) : Matrix of the initial state
    Returns:
        local_optimum (narray(n, n)) : Local maximum 
    """
    current_state = initial_state
    while True:
        best_neighbor, best_dist = None, float('inf')
        for neighbor in get_neighbors(current_state):
            if manhattan_dist(neighbor, goal_state) < best_dist:
                best_neighbor, best_dist = neighbor, manhattan_dist(neighbor, goal_state)

        current_dist = manhattan_dist(current_state, goal_state)
        if current_dist > best_dist:
            current_state = best_neighbor
        else:
            return current_state
            

In [30]:
# Solve the puzzles using hill climbing search
print("Problem\t\t\t\t    Initial Dist\tOptimal Solution\t\tFinal Dist")
print("-----------------------------------------------------------------------------------------------")
for problem in all_problems:
    solution = hill_climbing_search(problem)
    initial_dist = manhattan_dist(problem, goal_state)
    final_dist = manhattan_dist(solution, goal_state)
    print(f"{problem}\t{initial_dist}\t{solution}\t{final_dist}")

Problem				    Initial Dist	Optimal Solution		Final Dist
-----------------------------------------------------------------------------------------------
[[4, 2, 0], [5, 1, 3], [7, 8, 6]]	8	[[4, 2, 3], [5, 1, 6], [7, 8, 0]]	4
[[4, 1, 3], [0, 2, 6], [7, 5, 8]]	8	[[4, 1, 3], [0, 2, 6], [7, 5, 8]]	8
[[2, 0, 3], [8, 6, 5], [1, 4, 7]]	14	[[2, 0, 3], [8, 6, 5], [1, 4, 7]]	14
[[1, 2, 3], [4, 5, 0], [7, 8, 6]]	2	[[1, 2, 3], [4, 5, 6], [7, 8, 0]]	0
[[2, 3, 6], [1, 5, 8], [4, 7, 0]]	8	[[2, 3, 6], [1, 5, 8], [4, 7, 0]]	8
[[2, 3, 5], [1, 7, 6], [8, 4, 0]]	10	[[2, 3, 5], [1, 7, 6], [8, 4, 0]]	10
[[2, 1, 0], [3, 8, 6], [4, 7, 5]]	12	[[2, 1, 0], [3, 8, 6], [4, 7, 5]]	12
[[4, 1, 2], [7, 5, 3], [0, 8, 6]]	8	[[4, 1, 2], [7, 5, 3], [0, 8, 6]]	8
[[1, 3, 0], [8, 2, 5], [4, 7, 6]]	10	[[1, 3, 0], [8, 2, 5], [4, 7, 6]]	10
[[1, 5, 2], [4, 0, 3], [7, 8, 6]]	6	[[1, 5, 2], [4, 0, 3], [7, 8, 6]]	6
[[5, 0, 3], [2, 6, 8], [1, 4, 7]]	16	[[5, 0, 3], [2, 6, 8], [1, 4, 7]]	16
[[1, 0, 3], [4, 2, 5], [7, 8, 6]]	6	[[1, 2, 3